# Google Colab
Wir nutzen Colab, weil es eine einfache Programmieroberflaeche mit den benoetigten Bibliotheken bietet. Ausserdem gibt es GPUs kostenlos:

In [ ]:
! nvidia-smi

# Problemstellung

Welches Problem wollen wir lösen?
Die vielleicht wichtigste Frage beim Entwicklen eines Machine Learning Models ist: 

- Was ist das Problem?
- Was sind die Inputs?
- Was soll vorhergesagt werden?
- Was für eine Art Problem ist das?
  - Klassifizierung?
  - Regression?
  - Semantische Segmentierung?
  - Instanzsegmentierung?
  - Objekt-Erkennung?
  - Generierung?
  - ...?
- Wie messen wir ob unser Model gut ist?
  - Klassifizierung:
    - Genauigkeit
    - Sensitivitaet
    - Spezifizitaet
    - Konfusionsmatrix
    - F1-score
  - Regression:
    - Mean-squared error (MSE)
    - Mean absolute error (MAE)
  - Segmentierung:
    - Intersection-over-union (IoU)
    - Dice score (DSC)
  - und viele mehr...

## Ziffernerkennung

Was:
In diesem Beispiel wollen wir Checks automatisch auslesen:

![](https://blog.filestack.com/wp-content/uploads/2019/07/check-processing-1.png)

Nehmen wir mal an, dass wir schon wissen wo die Ziffern in das Datumsfeld und das Geldmenge-Feld eingetragen sind. Jetzt muessen wir die einzelnen Bilder von Ziffern in die jeweiligen Ziffern uebertragen:

![](https://miro.medium.com/max/1400/1*hVdoiW35FXUE-fZ0HI30Tw.jpeg)

Inputs: Bilder von Ziffern: $x$

Vorhersage: Wir sagen vorher, welche Ziffer ${0, 1, 2, 3, 4, 5, 6, 7, 8, 9}$ gesehen wurde.

Art des Problems: Klassifizierung in eine der 10 Ziffern - $p(\hat{y}=y\mid x)$

Testen: Wir messen wie viele Ziffern richtig vorhergesagt wurden - die "Genauigkeit" (= accuracy) unseres Klassifizierers:
$$acc = \frac{1}{n}\sum_{i=1}^{n} [(\arg \max_{\hat{y}}p(\hat{y}\mid x)) == y]$$





# Datensammlung

So, wo bekommen wir die Daten her? In echten Projekten wird einem teilweise ein Projekt vorgegeben und dann auch die Daten mitgeliefert. Manchmal denkt man sich aber selbst ein Projekt aus und muss selbst Daten sammeln oder suchen:
- https://www.tensorflow.org/datasets
- https://datasetsearch.research.google.com
- https://www.kaggle.com/datasets
- https://grand-challenge.org/challenges/

## MNIST
Wir haben Glueck -- der MNIST Datensatz: http://yann.lecun.com/exdb/mnist/

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/MnistExamples.png/320px-MnistExamples.png)

Hier haben wir 60.000 Training- und 10.000 Testbilder von Ziffern 0-9 und deren Klassenangehoerigkeit.

In [ ]:
# Wir koennen die Daten mit Tensorflow Datasets laden:
import tensorflow_datasets as tfds

(ds_train, ds_valid, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train[:70%]', 'train[70%:]', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Erste Datenanalyse

Als erstes sollten wir uns mit den Daten vertraut machen und einige Fragen beantworten:
- Wie sehen die Daten aus?
- Welches Format haben sie?
  - Wie groß sind die Bilder?
  - Welche Werte nehmen die Pixel an?
- Was sind die Statistiken:
  - Fuer Klassifizierungen, was ist die Klassenverteilung?
  - Fuer Regressionen, was ist die Verteilung der Werte? Was ist der Mittelwert, die Standardabweichung?

Fuer bekannte Datensaetze kann man auch https://knowyourdata-tfds.withgoogle.com benutzen.

In [ ]:
%matplotlib inline
# Schauen wir uns ein paar Daten an.
# Dazu nutzen wir tfds.visualization.show_examples:
help(tfds.visualization.show_examples)

# Das benutzt matplotlib zum anzeigen der Bilder.
# https://matplotlib.org
import matplotlib.pyplot as plt

# Und numpy brauchen wir spaeter um mit Matrizen und Vektoren zu arbeiten.
import numpy as np

tfds.visualization.show_examples(ds_train, ds_info)

plt.show()

# Die Beschriftungen sind "Klassenname (Klassen-ID)".

Wie wir hier sehen sind die Labels schon in Zahlen gespeichert und einfach zu verwenden. In anderen Datensaetzen wollen wir aber keine Ziffern klassifizieren sondern, z.B. Hunde und Katzen auseinander halten wollen. Dazu muessen wir diese Klassen auf numerische Kategorien umwandeln und numerieren dazu dann einfach die Klassen von $0-n$, wobei $n$ die Anzahl der Klassen ist. In den Datensaetzen die wir hier laden, wird dies automatisch gemacht. Falls ihr selbst Datensaetze gestaltet muesstet ihr diese Umwandlung selbst gestalten. 

Die Bilder sehen schon gut aus! Aber wie gross sind sie und wie steht es um die anderen Fragen?

In [ ]:
# ds_info enthaelt eine "Features"-Beschreibung wo man die Form der Bilder findet:
ds_info

In [ ]:
# Hier laden wir den Datensatz in ein Pandas DataFrame:
# ACHTUNG! Das funktioniert, weil MNIST recht klein ist - fuer groessere
# Datensaetze sollte man nur Teile laden.
# Deswegen dauert das auch ein Weilchen.
# https://pandas.pydata.org
import pandas as pd

df = tfds.as_dataframe(ds_train, ds_info)

# Pandas DataFrame sind Tabellen, wo wir eine Spalte mit dem Bild und eine mit
# dem Label haben:
df.head()

In [ ]:
# Wir koennen einzelne Bilder darstellen:
plt.imshow(df['image'][0].reshape((28, 28)))
# Wir muessen das Bild von (28, 28, 1) in (28, 28) unformen,
# da matplotlib das (..., ..., 1) nicht als Schwarz-Weiss versteht.
plt.show()

In [ ]:
# Und uns anschauen, welche Werte die Bilder annehmen:
print(f"Min: {df['image'][0].min()} - Max: {df['image'][0].max()}")

In [ ]:
# Wir koennen auch die Statistiken der Labels darstellen.
# Hierfuer ist Seaborn sehr nuetzlich:
# https://seaborn.pydata.org
import seaborn as sns
sns.countplot(data=df, x='label')
plt.show()

# Wir sehen, dass die Klassen halbwegs gleichmaessig verteilt sind - das ist
# relativ Wichtig fuer die Analyse spaeter!

# Bauen eines ersten Models

Jetzt, wo wir die Daten etwas verstanden haben, koennen wir unser erstes Model bauen!

Aber was optimieren wir eigentlich? Die Bernoulli-Likelihood ist nur fuer 2 Klassen gedacht...

Fuer mehrere Klassen benutzen wir die softmax-Funktion:
$$p(\hat{y}=i) = softmax(h)_i = \frac{exp(h_i)}{\sum_{j=1}^{k}exp(h_j))}$$

und die Kategorische (Categorical)-Likelihood:
$$p(y) = [y=1] * \pi_1 + \dots + [y=k] * \pi_k$$
Damit kann man die Likelihood unserer Messwerte berechnen als
$$\Rightarrow p(\hat{y}=y)=softmax(h)_y$$

## Datenvorbereitung

Wie wir gesehen haben, sind die Bildpixel im Bereich $[0, 255]$. Hier ist es hilfreich die Werte in den Bereich $[0, 1]$ oder $[-1, 1]$ zu konvertieren. Dies hilft, damit die Gewichte des neuronalen Netzes nicht so groß sein müssen. Dies wird angewandt mit `dataset.map(normalisierer)`.

Zusaetzlich speichern wir die geänderten Bilder im Arbeitsspeicher (`dataset.cache`), nutzt eine zufaellige Reihenfolge fuer die Bilder im Trainingsset (`dataset.shuffle`), und baut Batches mit Groesse 128 (`dataset.batch`). Zum Schluss nutzen wir noch "prefetching" (`dataset.prefetch`) um mehrere Batches im Speicher zu halten damit die spaeteren Berechnungen nicht darauf warten muessen.



In [ ]:
import tensorflow as tf

def normalize_img(image, label):
  """Normalisiert die Bilder von [0, 255] zu [0, 1] und konvertiert die Datentypen `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(int(ds_info.splits['train'].num_examples * 0.7))
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_valid = ds_valid.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_valid = ds_valid.cache()
ds_valid = ds_valid.batch(128)
ds_valid = ds_valid.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.cache()
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

## Modelerstellung

Jetzt kommen wir endlich zu dem Punkt an dem wir unser Model bauen! Dies besteht aus dem eigentlichen neuronalen Netz und der Loss-Funktion um zu bestimmen wie gut unser Model ist.

Wir benutzen die Keras-Bibliothek um die Modelle zu trainieren. Bei Keras gehoert auch der Optimierer mit in das Modell - Hier benutzen wir direkt den Adam-Optimierer.

In [ ]:
# Unser Model ist einfach die Aneinanderschachtelung von 2 Linearen-Layern mit
# einer ReLU-Funktion dazwischen. Hier fangen wir damit an das Bild von einer 
# 28x28 Matrix in einen 784 Vektor umzuformen (Flatten) und fuegen dann die
# Linear (hier Dense) Layers hinzu. Der "Hidden Layer" hat 128 Neuronen - das
# sind 784 x 128 Gewichte. Das letze Layer hat dann 128x10 Gewichte
# mit 10 Ausgangsneuronen, die die Wahrscheinlichkeiten der verschiedenen
# Klassen modellieren.
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])


In [ ]:
# Anschliessend stellen wir unser Model fertig, indem wir einen Optimierer
# (Adam), die Loss-Funktion und Metriken, die wir messen wollen, festlegen.
# Hier nutzen wir die `SparseCategoricalCrossentropy` Loss-Funktion. Diese ist
# eine Kategorische (Categorical) Likelihood - sie ist "Sparse", weil die
# Labels als Klassen-ID (0-9) angegeben anstatt als Wahrscheinlichkeitsvektoren:
# 1 = [1, 0, 0, ..., 0], 2 = [0, 1, 0, 0, ..., 0].
# Ausserdem nutzen wir logits - dies bedeutet, dass unser Netzwerk den Input in
# den Softmax ausgibt anstatt die Wahrscheinlichkeiten nach dem Softmax.
# Als Metriken wollen wir einfach nur die Genauigkeit (Accuracy) messen.
# Fuer den Adam-Optimierer nutzen wir eine learning rate von 0.001.

# Uebrigens - `compile` legt auch die ersten "weights" fest.
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

## Modeltraining

Das Model muss jetzt nur noch trainiert werden und dann waeren wir auch schon fertig. Keras macht uns das sehr einfach -- wir muessen nur `model.fit` aufrufen und die Daten festlegen:

In [ ]:
# Hier nutzen wir `ds_train` als Trainingsdatensatz, trainieren fuer 10 Epochen
# und nutzen `ds_valid` als Validierungsdatensatz.
model.fit(
    ds_train,
    epochs=15,
    validation_data=ds_valid,
)

# Analyse der Ergebnisse

Nachdem wir jetzt unser Modell trainiert haben koennen wir das Model auch auf dem Testdaten ausprobieren. Hierzu bietet Keras wieder eine einfache Funktion: `model.evaluate`

In [ ]:
print(model.evaluate(ds_train))
print(model.evaluate(ds_valid))
print(model.evaluate(ds_test))

Wir sehen hier schon, dass wir 97% Genauigkeit erreichen koennen - welche Zahlen bekommen wir aber besonders gut hin und welche weniger?

In [ ]:
%matplotlib inline
# Hier benutzen wir sklearn um Konfusionsmatrizen zu erstellen und generelle
# Berichte zu der Performance zu verfassen.
# https://scikit-learn.org/stable/index.html
from sklearn.metrics import classification_report, confusion_matrix

# Hier benutzen wir `model.predict` um die Vorhersagen zu erhalten.
# Leider haben unsere Optimierungen am Datensatz, dass wir batchen und prefetchen
# dazu gefuehrt dass wir die Reihenfolge der Daten nicht mehr kennen. Also speichern
# wir die Eingabewerte beim Vorhersagen:

y_prob = []  # Hier speichern wir die Modelvorhersagen
y_true = []  # Hier speichern wir die echten Labels
x_input = [] # Hier speichern wir die Bilder

# Wir sagen die Ergebnisse unseres Modells auf den Validierungsdaten her.
for image_batch, label_batch in ds_valid:
   # Speichere die Eingaben
   x_input.append(image_batch.numpy())
   y_true.append(label_batch.numpy())
   # Berechne die Vorhersagen
   preds = model.predict(image_batch)
   # Hier nutzen wir den Softmax um die Wahrscheinlichkeiten zu berechnen und 
   # speichern diese in y_prob.
   y_prob.append(tf.nn.softmax(preds).numpy())

# Zum Schluss wandeln wir unseren Speicher noch in Matrizen um:
x_input = np.concatenate(x_input)
y_true = np.concatenate(y_true)
y_prob = np.concatenate(y_prob)

y_pred = np.argmax(y_prob, axis=1)

# sklearn bietet eine einfach Funktion fuer Konfusionsmatrizen:
cm = confusion_matrix(y_true, y_pred)

# die wir nur noch darstellen muessen.
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, annot_kws={"size": 8}, cmap="YlGnBu") # font size
plt.show()

# Und dann koennen wir uns einen Bericht ueber die Ergebnisse der einzelnen
# Klassen ansehen.
print(classification_report(y_true, y_pred))

Um das ganze uns noch genauer anzuschauen koennen wir versuchen zu verstehen weshalb manche Ziffern falsch kategorisiert werden:

In [ ]:
y_prob_class = y_prob[np.arange(len(y_prob)), y_true]

low_prob_samples = np.argsort(y_prob_class)

for idx in low_prob_samples[:5]:
  pred_prob = y_prob[idx]
  pred_label = np.argmax(pred_prob)
  print(f'[{idx}] Vorhersage:\n{pred_prob}\n-> {pred_label} -- Echt: {y_true[idx]}')
  plt.imshow(x_input[idx].reshape(28, 28))
  plt.show()

Hier haben wir gesehen, dass unser Modell schon sehr gut ist und die einzigen Fehler eigentlich Ziffern sind, die schwer zu erkennen sind. Manchmal kann man damit aber auch Fehler im Datensatz finden.

# Verbessern des Models

Wie wir gesehen haben, hat dieses Model noch einige Probleme. Stattdessen können wir ein Convolutional Neural Network (CNN) bauen, was besser fuer Bilder ist.

Ausserdem benutzen wir Max-Pooling um die Groesse der Zwischenschritte zu verkleinern und Speicherplatz zu sparen:
![](https://cs231n.github.io/assets/cnn/maxpool.jpeg)

In [ ]:
# Hier nutzen wir 2 convolutional Layer mit ReLU-Aktivierung
# und Max-Pooling danach.
# Nach den CNN-Layern, aendern wir wieder die Form in einen Vektor (`flatten`)
# und haben noch zwei "normale" Layer.
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, 3, activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Conv2D(16, 3, activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu',
                        kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
  tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(1e-3))
])

# Der Rest ist das Gleiche wie vorher!
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_valid,
)

In [ ]:
# Und wir sehen, dass unsere Validierungs- und Testergebnisse besser sind
# und wir nicht mehr overfitten:
print(model.evaluate(ds_train))
print(model.evaluate(ds_valid))
print(model.evaluate(ds_test))

# Keras Modelle
Keras bietet aber auch schon vorgefertigte Modelle an, die veroeffentlicht wurden: https://keras.io/api/applications/

Hier koennen wir zum Beispiel ein ResNet (residual network mit diesen Sprungverbindungen) einfach laden und benutzen:

In [ ]:
input_layer = tf.keras.layers.Input((28, 28, 1))
padded = tf.keras.layers.ZeroPadding2D(padding=(2, 2))(input_layer)
model = tf.keras.applications.ResNet50(
      include_top=True,
      input_tensor=padded,
      pooling="avg",
      weights=None,
      classes=10,
  )

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_valid,
)

In [ ]:
# Der MNIST Datensatz ist relativ klein, deswegen sehen wir keinen grossen
# Vorteil mit diesem groesseren Model! Ausserdem muessten wir vmtl laenger
# trainieren...
print(model.evaluate(ds_train))
print(model.evaluate(ds_valid))
print(model.evaluate(ds_test))